In [1]:
import numpy as np
import scipy.io
import scipy.linalg
import sklearn.metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

import anndata as ad
import scanpy as sc
from sklearn.neighbors import KDTree
import igraph as ig
from scipy import sparse
from sklearn.neighbors import KDTree
from itertools import chain

import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def kernel(ker, X1, X2, gamma):
    K = None
    if not ker or ker == 'primal':
        K = X1
    elif ker == 'linear':
        if X2 is not None:
            K = sklearn.metrics.pairwise.linear_kernel(
                np.asarray(X1).T, np.asarray(X2).T)
        else:
            K = sklearn.metrics.pairwise.linear_kernel(np.asarray(X1).T)
    elif ker == 'rbf':
        if X2 is not None:
            K = sklearn.metrics.pairwise.rbf_kernel(
                np.asarray(X1).T, np.asarray(X2).T, gamma)
        else:
            K = sklearn.metrics.pairwise.rbf_kernel(
                np.asarray(X1).T, None, gamma)
    return K


class TCA:
    def __init__(self, kernel_type='primal', dim=30, lamb=1, gamma=1):
        '''
        Init func
        :param kernel_type: kernel, values: 'primal' | 'linear' | 'rbf'
        :param dim: dimension after transfer
        :param lamb: lambda value in equation
        :param gamma: kernel bandwidth for rbf kernel
        '''
        self.kernel_type = kernel_type
        self.dim = dim
        self.lamb = lamb
        self.gamma = gamma

    def fit(self, Xs, Xt):
        '''
        Transform Xs and Xt
        :param Xs: ns * n_feature, source feature
        :param Xt: nt * n_feature, target feature
        :return: Xs_new and Xt_new after TCA
        '''
        X = np.hstack((Xs.T, Xt.T))
        X /= np.linalg.norm(X, axis=0)
        m, n = X.shape
        ns, nt = len(Xs), len(Xt)
        e = np.vstack((1 / ns * np.ones((ns, 1)), -1 / nt * np.ones((nt, 1))))
        M = e * e.T
        M = M / np.linalg.norm(M, 'fro')
        H = np.eye(n) - 1 / n * np.ones((n, n))
        K = kernel(self.kernel_type, X, None, gamma=self.gamma)
        n_eye = m if self.kernel_type == 'primal' else n
        a, b = K @ M @ K.T + self.lamb * np.eye(n_eye), K @ H @ K.T
        w, V = scipy.linalg.eig(a, b)
        ind = np.argsort(w)
        A = V[:, ind[:self.dim]]
        Z = A.T @ K
        Z /= np.linalg.norm(Z, axis=0)

        Xs_new, Xt_new = Z[:, :ns].T, Z[:, ns:].T
        return Xs_new, Xt_new

In [ ]:
# covid19 mdm
# load data
print('load data...')
sc_data_processed = pd.read_csv('~/workspace/spatial_cluster/covid_19/sc_data_processed.csv', index_col=0)
sc_meta_processed = pd.read_csv('~/workspace/spatial_cluster/covid_19/sc_meta_processed.csv', index_col=0)
st_data_processed = pd.read_csv('~/workspace/spatial_cluster/covid_19/st_data_processed.csv', index_col=0)
st_meta_processed = pd.read_csv('~/workspace/spatial_cluster/covid_19/st_meta_processed.csv', index_col=0)

# tca
print('tca...')
obj = TCA(kernel_type='primal', dim=50, lamb=1, gamma=1)
sc_new, st_new = obj.fit(Xs=np.array(sc_data_processed.T), Xt=np.array(st_data_processed.T))
print('tca done.')

load data...
tca...


In [7]:
print('mlp...')
label = np.array(st_meta_processed[['V3','V4']])
input_size = st_new.shape[1]
hidden_size = 128
output_size = 2
batch_size = 16
my_nn = torch.nn.Sequential(
    torch.nn.Linear(input_size, hidden_size),
    torch.nn.Sigmoid(),
    torch.nn.Linear(hidden_size, output_size),
)
cost = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(my_nn.parameters(), lr = 0.001)

losses = []
for i in range(1000):
    batch_loss = []
    for start in range(0, len(st_new), batch_size):
        end = start + batch_size if start + batch_size < len(st_new) else len(st_new)
        xx = torch.tensor(st_new[start:end], dtype = torch.float, requires_grad = True)
        yy = torch.tensor(label[start:end], dtype = torch.float, requires_grad = True)
        prediction = my_nn(xx)
        loss = cost(prediction, yy)
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()
        batch_loss.append(loss.data.numpy())
        
    if i % 100==0:
        losses.append(np.mean(batch_loss))
        print(i, np.mean(batch_loss))

print('saving...')        
predict = my_nn(torch.tensor(np.array(sc_new), dtype = torch.float, requires_grad = True)).data.numpy()
pd.DataFrame(predict).to_csv('~/workspace/spatial_cluster/covid_19/pseudo_coords.csv')

mlp...
0 2169.201
100 148.96654
200 126.64259
300 113.56344
400 104.2271
500 96.40067
600 89.60366
700 83.597404
800 78.12729
900 73.05979
saving...
